# NeuroFetal AI - Enhanced Fusion ResNet Training

**Version 2.0** - With Advanced Enhancements:
- 3-Input Architecture (FHR + Tabular + CSP)
- Squeeze-and-Excitation (SE) Blocks
- Multi-Head Self-Attention
- 19 CSP Features
- Focal Loss for Class Imbalance

### Instructions:
1. **Runtime -> Change runtime type -> GPU (T4)**
2. Run cells in order
3. Results save to `Reports/training_logs/`

## GitHub Authentication
Enter your Personal Access Token when prompted (not saved to notebook).

In [ ]:
from getpass import getpass
import os

GITHUB_REPO = "Krishna200608/NeuroFetal-AI"
GITHUB_TOKEN = getpass("GitHub Personal Access Token: ")

os.environ['GITHUB_TOKEN'] = GITHUB_TOKEN
os.environ['GITHUB_REPO'] = GITHUB_REPO
print("Token saved to session.")

## Step 0: Clone Repository
Dataset is included in the repo at `Datasets/ctu_uhb_data/`

In [ ]:
import os

GITHUB_REPO = os.environ.get('GITHUB_REPO', 'Krishna200608/NeuroFetal-AI')
GITHUB_TOKEN = os.environ.get('GITHUB_TOKEN', '')
BRANCH = "main"

if not os.path.exists("/content/NeuroFetal-AI"):
    if GITHUB_TOKEN:
        !git clone --branch {BRANCH} https://{GITHUB_TOKEN}@github.com/{GITHUB_REPO}.git /content/NeuroFetal-AI
    else:
        !git clone --branch {BRANCH} https://github.com/{GITHUB_REPO}.git /content/NeuroFetal-AI
    print("Repository cloned!")
else:
    os.chdir("/content/NeuroFetal-AI")
    !git pull origin {BRANCH}
    print("Repository updated!")

In [ ]:
import os
import sys

PROJECT_ROOT = "/content/NeuroFetal-AI/"
CODE_DIR = os.path.join(PROJECT_ROOT, "Code")
SCRIPTS_DIR = os.path.join(CODE_DIR, "scripts")
UTILS_DIR = os.path.join(CODE_DIR, "utils")
DATASET_DIR = os.path.join(PROJECT_ROOT, "Datasets", "ctu_uhb_data")

os.chdir(SCRIPTS_DIR)
sys.path.insert(0, SCRIPTS_DIR)
sys.path.insert(0, UTILS_DIR)
sys.path.insert(0, CODE_DIR)

print(f"Working dir: {os.getcwd()}")
print(f"Dataset: {len([f for f in os.listdir(DATASET_DIR) if f.endswith('.dat')])} records")

## Step 1: Install Dependencies

In [ ]:
!pip install -q wfdb shap scipy

In [ ]:
import tensorflow as tf
print(f"TensorFlow: {tf.__version__}")
print(f"GPU: {tf.config.list_physical_devices('GPU')}")

## Step 2: Data Ingestion

In [ ]:
processed_dir = os.path.join(PROJECT_ROOT, "Datasets", "processed")
if os.path.exists(os.path.join(processed_dir, "X_fhr.npy")):
    print("Processed data exists. Skipping.")
else:
    print("Running data ingestion...")
    !python data_ingestion.py

In [ ]:
import numpy as np

X_fhr = np.load(os.path.join(processed_dir, "X_fhr.npy"))
X_tabular = np.load(os.path.join(processed_dir, "X_tabular.npy"))
y = np.load(os.path.join(processed_dir, "y.npy"))

print(f"FHR: {X_fhr.shape}, Tabular: {X_tabular.shape}, Labels: {y.shape}")
print(f"Class Balance: {np.mean(y):.2%} positive")

## Step 3: Model Architecture

In [ ]:
from model import build_enhanced_fusion_resnet

model = build_enhanced_fusion_resnet(
    input_shape_fhr=(X_fhr.shape[1], 1),
    input_shape_tabular=(X_tabular.shape[1],),
    input_shape_csp=(19,),
    use_se_blocks=True,
    use_attention=True
)
print(f"Parameters: {model.count_params():,}")

## Step 4: Training

In [ ]:
!python train.py

## Step 5: Explainability (XAI)

In [ ]:
!python xai.py

## Step 6: Push Results to GitHub

In [ ]:
!git config --global user.email "krishnabhujel2006@gmail.com"
!git config --global user.name "Krishna Bhujel"

In [ ]:
os.chdir(PROJECT_ROOT)

!git add Code/models/*.keras Reports/training_logs/*.json Code/figures/*.png 2>/dev/null || true
!git status
!git commit -m "Add trained model from Colab" || echo "Nothing to commit"
!git push origin main

## Results

In [ ]:
import json, glob

log_dir = os.path.join(PROJECT_ROOT, "Reports", "training_logs")
logs = sorted(glob.glob(os.path.join(log_dir, "training_log_*.json")))

if logs:
    with open(logs[-1]) as f:
        r = json.load(f)
    print(f"Mean AUC: {r['summary']['mean_auc']:.4f} +/- {r['summary']['std_auc']:.4f}")
else:
    print("No training logs found.")